# Core Update

This Notebook focuses on the process of modifying data.


The following topics are covered:
- Joins
- Subquery
- In


Further Reading:
- [UPDATE syntax](https://www.sqlite.org/lang_update.html) by SQLite
- ["Using UPDATE and DELETE Statements"](https://docs.sqlalchemy.org/en/20/tutorial/data_update.html) by SQLAlchemy